In [2]:

import numpy as np
from random import randrange
import qiskit as qk
from qiskit import Aer
from qiskit import execute
import math
import sys
import random
import time
from tqdm import tqdm
import torch
# ========================
# Preformatting Start
# ========================

# Generate a list of array size binaries
def generateStates(size):
    state = []
    sizeRoot = int(math.log(size,2))
    for i in range(size):
        state.append(bin(i))
    for i in range(size):
        state[i] = state[i][1:]
        state[i] = state[i].replace('b', '')
        if len(state[i]) < sizeRoot: state[i] = (add0(sizeRoot - (len(state[i])))) + state[i]
    return state


def markStates(state, matrix):
    size = len(matrix)
    for i in range(size):
        if matrix[i] == 1:
            state[i] = '-' + state[i]
        else:
            state[i] = '+' + state[i]
    return state


def add0(n):
    if n == 0:
        return ('')
    else:
        return ('0' + (add0(n - 1)))


def allPositive(state, size):
    for i in range(size):
        state[i] = '+' + state[i]
    return state



def howMany1(string):
    count = 0
    for i in range(len(string)):
        if string[i] == '1': count = count + 1
    return count

def singFlip(string):
    if string[0] == '-':
        return ('+' + string[1:])
    else:
        return ('-' + string[1:])


def whereIs1(string):
    locations = ''
    string = string[1:]
    for i in range(len(string)):
        if string[i] == '1': locations = locations + str(i)
    return locations


def flipSingOn(state, location, maxIndex):
    for i in range(maxIndex):
        if state[i][location] == '1':
            state[i] = singFlip(state[i])
    return state


# singFlip em todos os elementos com '1' em duas posições especificas
def flipSingOn2(state, qubits, maxIndex):
    q0 = int(qubits[0]) + 1
    q1 = int(qubits[1]) + 1
    for i in range(maxIndex):
        if state[i][q0] == '1' and state[i][q1] == '1':
            state[i] = singFlip(state[i])
    return state


# singFlip em todos os elementos com '1' em três posições especificas
def flipSingOn3(state, qubits, maxIndex):
    q0 = int(qubits[0]) + 1
    q1 = int(qubits[1]) + 1
    q2 = int(qubits[2]) + 1
    for i in range(maxIndex):
        if state[i][q0] == '1' and state[i][q1] == '1' and state[i][q2] == '1':
            state[i] = singFlip(state[i])
    return state


# singFlip em todos os elementos com '1' em quatro posições especificas
def flipSingOn4(state, qubits, maxIndex):
    q0 = int(qubits[0]) + 1
    q1 = int(qubits[1]) + 1
    q2 = int(qubits[2]) + 1
    q3 = int(qubits[3]) + 1
    for i in range(maxIndex):
        if state[i][q0] == '1' and state[i][q1] == '1' and state[i][q2] == '1' and state[i][q3] == '1':
            state[i] = singFlip(state[i])
    return state


# Generates circuit for 1 | 1⟩ bits
def runTo1(circuit,statusVector, n, q, maxIndex):
    # print("*"*40,"in run to 1","*"*40)
    # print_status(statusVector,"status_in_run_1-1")
    theQubits = whereIs1(statusVector[n])
    # print(n,statusVector[n],theQubits,type(theQubits))
    circuit.z(q[int(theQubits[0])])  # apply z to qubit
    # print(circuit)
    # multiplies by -1 on the status vector
    statusVector = flipSingOn(statusVector, int(theQubits[0]) + 1, maxIndex)
    # print_status(statusVector,"status_in_run_1-2")

    return statusVector


# Gera circuito para bits com 2 |1⟩
def runTo2(circuit,statusVector, n, q, maxIndex):
    # print("*"*40,"runTo2","*"*40)
    theQubits = whereIs1(statusVector[n])
    # print_status(statusVector,"status_run2")
    # print(n,statusVector[n],theQubits)
    q0 = int(theQubits[0])
    q1 = int(theQubits[1])
    circuit.cz(q[q0], q[q1])
    # print(circuit)
    statusVector = flipSingOn2(statusVector, theQubits, maxIndex)
    # print_status(statusVector,"status_run2")
    return statusVector


# Gera circuito para bits com 3 |1⟩
def runTo3(circuit,statusVector, n, q, aux, maxIndex):
    theQubits = whereIs1(statusVector[n])
    q0 = int(theQubits[0])
    q1 = int(theQubits[1])
    q2 = int(theQubits[2])
    
    circuit = ccz(circuit, q[q0], q[q1], q[q2], aux[0])
    statusVector = flipSingOn3(statusVector, theQubits, maxIndex)
    return statusVector


# Gera circuito para bits com 4 |1⟩
def runTo4(circuit,statusVector, n, q, aux, maxIndex):
    theQubits = whereIs1(statusVector[n])
    q0 = int(theQubits[0])
    q1 = int(theQubits[1])
    q2 = int(theQubits[2])
    q3 = int(theQubits[3])
    
    circuit = cccz(circuit, q[q0], q[q1], q[q2], q[q3], aux[0], aux[1])
    statusVector = flipSingOn4(statusVector, theQubits, maxIndex)
    return statusVector


# =======================
# MULTI-CONTROLLED GATES
# =======================

def ccz(circ, q1, q2, q3, aux1):
    # Apply Z-gate to a state controlled by 3 qubits
    circ.ccx(q1, q2, aux1)
    circ.cz(aux1, q3)
    # cleaning the aux bit
    circ.ccx(q1, q2, aux1)
    return circ


def cccz(circ, q1, q2, q3, q4, aux1, aux2):
    # Apply Z-gate to a state controlled by 4 qubits
    circ.ccx(q1, q2, aux1)
    circ.ccx(q3, aux1, aux2)
    circ.cz(aux2, q4)
    # cleaning the aux bits
    circ.ccx(q3, aux1, aux2)
    circ.ccx(q1, q2, aux1)
    return circ

def cccx(circ, q1, q2, q3, q4, aux1):
    circ.ccx(q1, q2, aux1)
    circ.ccx(q3, aux1, q4)    
    # cleaning the aux bits    
    circ.ccx(q1, q2, aux1)
    return circ


def ccccx(circ, q1, q2, q3, q4, q5, aux1, aux2):
    circ.ccx(q1, q2, aux1)
    circ.ccx(q3, q4, aux2)
    circ.ccx(aux2, aux1, q5)
    # cleaning the aux bits
    circ.ccx(q3, q4, aux2)
    circ.ccx(q1, q2, aux1)
    return circ

def ccccccx(circ, q, q_out, aux):
    circ.ccx(q[0], q[1], aux[0])
    circ.ccx(q[2], q[3], aux[1])
    circ.ccx(q[4], q[5], aux[2])
    circ.ccx(aux[0], aux[1], aux[3])
    circ.ccx(aux[2], aux[3], q_out)
    # cleaning the aux bits
    circ.ccx(aux[0], aux[1], aux[3])
    circ.ccx(q[4], q[5], aux[2])
    circ.ccx(q[2], q[3], aux[1])
    circ.ccx(q[0], q[1], aux[0])
    return circ



def print_status(vector, name="status_vector"):
    print("=" * 40, name, "=" * 40)
    for v in vector:
        print(v, end=" ")
    print()
    print("-" * 100)


# Gera circuito para operador U, baseado em sua matriz representativa
def generateU(circuit,matrix, q, aux):
    maxIndex = len(matrix)
    goalVector = generateStates(maxIndex)
    goalVector = markStates(goalVector, matrix)
    # print_status(goalVector,"goal_vector")
    statusVector = generateStates(maxIndex)
    statusVector = allPositive(statusVector, maxIndex)

    # Checking if goalVector[0] is -1
    if goalVector[0][0] == '-':
        for i in range(maxIndex):
            goalVector[i] = singFlip(goalVector[i])

    # print_status(goalVector, "goal_vector")
    # Weiwen: Add Z gate
    for n in range(maxIndex):
        if (statusVector[n] != goalVector[n]) and (howMany1(goalVector[n]) == 1):
            # print(n,end=" ")
            statusVector = runTo1(circuit,statusVector, n, q, maxIndex)
    # print()
    # print_status(statusVector)

    # Weiwen: Add cZ on two qbits whose sign is not consistent with the goal
    for n in range(maxIndex):
        if (statusVector[n] != goalVector[n]) and (howMany1(goalVector[n]) == 2):
            statusVector = runTo2(circuit,statusVector, n, q, maxIndex)

    # print(circuit)
    # print_status(statusVector)

    for n in range(maxIndex):
        if (statusVector[n] != goalVector[n]) and (howMany1(goalVector[n]) == 3):
            statusVector = runTo3(circuit,statusVector, n, q, aux, maxIndex)

    # print(circuit)
    # print_status(statusVector)

    for n in range(maxIndex):
        if (statusVector[n] != goalVector[n]) and (howMany1(goalVector[n]) == 4):
            statusVector = runTo4(circuit,statusVector, n, q, aux, maxIndex)
    # print(circuit)
    # print_status(statusVector)

    # circuit.barrier()

def encoder2_1(circuit, q, o):
    circuit.cx(q[0], q[1])
    circuit.h(o)

    circuit.cz(q[1], o)

    circuit.barrier()
    return o

def encoder4_2(circuit, q, aux):
    for i in range(1,4):    
        circuit.cx(q[0],q[i])
        
    for i in [4,5]:
        circuit.h(q[i])    
        
    circuit.cz(q[1],q[5])
    ccz(circuit,q[1],q[5],q[4],aux[0])
        
    circuit.cz(q[2],q[4])
    ccz(circuit,q[2],q[4],q[5],aux[0])
        
    ccz(circuit,q[3],q[4],q[5],aux[0])
    
    circuit.barrier()
    return q[4:6]

def encoder8_3(circuit, q, aux):
    for i in range(1,8):    
        circuit.cx(q[0],q[i])
        
    for i in [8,9,10]:
        circuit.h(q[i])    
        
    circuit.cz(q[1],q[10])
    ccz(circuit,q[1],q[10],q[9],aux[0])
    ccz(circuit,q[1],q[10],q[8],aux[0])
    cccz(circuit,q[1],q[10],q[9],q[8],aux[0],aux[1])
    
    circuit.cz(q[2],q[9])
    ccz(circuit,q[2],q[9],q[8],aux[0])
    ccz(circuit,q[2],q[9],q[10],aux[0])
    cccz(circuit,q[2],q[9],q[8],q[10],aux[0],aux[1])
    
    ccz(circuit,q[3],q[9],q[10],aux[0])
    cccz(circuit,q[3],q[9],q[10],q[8],aux[0],aux[1])
    
    circuit.cz(q[4],q[8])
    ccz(circuit,q[4],q[8],q[10],aux[0])
    ccz(circuit,q[4],q[8],q[9],aux[0])
    cccz(circuit,q[4],q[8],q[9],q[10],aux[0],aux[1])
    
    ccz(circuit,q[5],q[8],q[10],aux[0])
    cccz(circuit,q[5],q[8],q[9],q[10],aux[0],aux[1])
    
    ccz(circuit,q[6],q[8],q[9],aux[0])
    cccz(circuit,q[6],q[8],q[9],q[10],aux[0],aux[1])
    
    cccz(circuit,q[7],q[8],q[9],q[10],aux[0],aux[1])

    circuit.barrier()
    return q[8:11]

from qiskit.tools.monitor import job_monitor

def simulate(circuit,shots,iter,printable=True):
    if printable:
        print(circuit)
    
    count_set = []
    
    for it in range(iter):        
        backend = Aer.get_backend('qasm_simulator')
        job_sim = execute(circuit, backend, shots=shots)
        job_monitor(job_sim)
        result_sim = job_sim.result()
        counts = result_sim.get_counts()
        count_set.append(counts)
    

    return count_set



def single_priceptro(circuit,q,input,weights):
    
    
    for i in range(4):
        circuit.h(q[i])
    generateU(input, q, aux)
    circuit.barrier()
    generateU(weights,q,aux)
    circuit.barrier()
    for i in range(4):
        circuit.h(q[i])
        circuit.x(q[i])
    circuit = ccccx(circuit, q[0], q[1], q[2], q[3], q[4], aux[0], aux[1])
    # circuit.measure(q[4], c[0])
    circuit.barrier()
    
    
def reset_qbits(q_set):
    for q in q_set:
        circuit.reset(q)
    circuit.barrier()



# MINST Circuit Generate


def init(circuit, input, work):  
   
    for idx in range(len(input)):
        
        # if input[idx]<0.5:
        #     circuit.x(work[idx])
        y_v = input[idx].item()
        if y_v > 0:
            alpha = np.arccos(y_v)
        elif y_v < 0:
            alpha = np.pi - np.arccos(-y_v)
        else:
            alpha = np.pi/2
        circuit.ry(alpha,work[idx])
    circuit.barrier()
    


def reset_qbits(q_set):
    for q in q_set:
        circuit.reset(q)
    circuit.barrier()


def SLP_4_encoding(circuit, q_in, q_en, input=[]):
    if len(input)!=0:        
        init(circuit,input,q_in)
        
    encoder_q_set = []
    for q in q_in:
        encoder_q_set.append(q)
    for q in q_en:
        encoder_q_set.append(q)
    encoder4_2(circuit,encoder_q_set,aux)

def SLP_4_Uw(circuit, q_en, q_out, w):
    
    beg = len(circuit.data)
    generateU(circuit, w,q_en,aux)
    end = len(circuit.data)
    
    circuit.barrier()
    for qbit in q_en[0:2]:
        circuit.h(qbit)
        circuit.x(qbit)
   
    circuit.ccx(q_en[0], q_en[1], q_out)
    circuit.barrier()
  
    return beg,end
def SLP_8_encoding(circuit, q_in, q_en, input=[]):
    if len(input)!=0:
        init(circuit, input, q_in)

    encoder_q_set = []
    for q in q_in:
        encoder_q_set.append(q)
    for q in q_en:
        encoder_q_set.append(q)
    encoder8_3(circuit,encoder_q_set,aux)
    
def SLP_8_Uw(circuit, q_en, q_out, w):    
    beg = len(circuit.data)
    generateU(circuit, w,q_en,aux)
    end = len(circuit.data)
    
    # circuit.barrier()
    # for qbit in q_en[0:3]:
    #     circuit.h(qbit)
    #     circuit.x(qbit)
    # 
    # cccx(circuit, q_en[0], q_en[1], q_en[2], q_out, aux[0])
    # circuit.barrier()
  
    return beg,end

def reverse_part_circuit(circuit,beg,end):
    
    for inst, qargs, cargs in reversed(circuit.data[beg:end]):
        circuit.data.append((inst.inverse(),qargs,cargs))


In [ ]:
Input =  [-1., -1., -1.,  1.,  1.,  1., -1., -1., 
         -1., -1., -1.,  1.,  1.,  1., -1., -1., 
         -1., -1.,  1.,  1.,  1., -1., -1., -1., 
         -1., -1.,  1.,  1.,  1.,  1.,  1., -1., 
         -1., -1.,  1.,  1.,  1.,  1.,  1., -1., 
         -1., -1.,  1.,  1.,  1.,  1.,  1., -1., 
         -1., -1.,  1.,  1.,  1.,  1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1.]
W = [-1., -1.,  1., -1., -1., -1., -1., -1., -1.,  1.,  1., -1.,  1.,  1.,
         -1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1.,
          1., -1., -1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1.,  1.,
         -1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1., -1., -1.,
         -1.,  1.,  1., -1.,  1., -1.,  1.,  1.]



maxIndex = len(Input)
        



q_en = qk.QuantumRegister(6,"encoded")
q_out = qk.QuantumRegister(1,"output")
c = qk.ClassicalRegister(1,"reg")
aux = qk.QuantumRegister(4,"aux")

circuit = qk.QuantumCircuit(q_en, q_out, aux, c)

# SLP_8_encoding(circuit, q_io, q_en, input)
# reset_qbits(q_io)

for qbit in q_en[0:6]:
    circuit.h(qbit)

circuit.barrier()
SLP_8_Uw(circuit, q_en, q_out[0], Input)
circuit.barrier()
SLP_8_Uw(circuit, q_en, q_out[0], W)

# reset_qbits(q_en)
circuit.barrier()
for qbit in q_en[0:6]:
    circuit.h(qbit)
    circuit.x(qbit)
ccccccx(circuit,q_en[0:6],q_out,aux)

circuit.barrier()
circuit.measure(q_out,c)
 
print(circuit)

In [14]:
def do_slp_via_th(input_ori,w_ori):
    p = input_ori
    d = 4*p*(1-p)
    e = (2*p-1)
    # e_sq = torch.tensor(1)
    w = w_ori
    
    sum_of_sq = (d+e.pow(2)).sum(-1)
    sum_of_sq = sum_of_sq.unsqueeze(-1)        
    sum_of_sq = sum_of_sq.expand(p.shape[0], w.shape[0])
            
    diag_p = torch.diag_embed(p)        
    
    p_w = torch.matmul(w,diag_p)
    
    z_p_w = torch.zeros_like(p_w)        
    shft_p_w = torch.cat((p_w, z_p_w), -1)
    
    sum_of_cross = torch.zeros_like(p_w)
    length = p.shape[1]    
    
    for shft in range(1,length):    
        sum_of_cross += shft_p_w[:,:,0:length]*shft_p_w[:,:,shft:length+shft]

    sum_of_cross = sum_of_cross.sum(-1)
            
    return (sum_of_sq+2*sum_of_cross)/(length**2) 



In [ ]:

qc_shots = 10000
num_c_reg = 4

print("="*50)
print("Start simulation:")
start = time.time()        
iters = 1
counts = simulate(circuit,qc_shots,iters,False)

end = time.time()
qc_time = end - start


print("From QC:",counts)
print("Simulation elasped time:",qc_time)
    
def analyze(counts):
    mycount = {}
    for i in range(num_c_reg):
        mycount[i] = 0
    for k,v in counts.items():
        bits = len(k) 
        for i in range(bits):            
            if k[bits-1-i] == "1":
                if i in mycount.keys():
                    mycount[i] += v
                else:
                    mycount[i] = v
    return mycount,bits

# for k,v in counts[0].items():
#     print(k,v)
(mycount,bits) = analyze(counts[0])

for b in range(bits):
    print (b,float(mycount[b])/qc_shots)
    


print("="*100)
import torch
print("Theoretic Analysis")

print(do_slp_via_th(torch.tensor([Input]),torch.tensor([W])))

In [ ]:
import torch

input_ori = [0.0,0.95,0.05,0.1,0.99,0.01,0.0,1.0]
# input_ori = [0,1,0,0,1,0,0,1]
input = torch.tensor(input_ori)*2-1



w1    = [ -1,  1, -1, -1,   1,  -1, -1,  1]
w2    = [ -1,  1,  -1, -1,   1,  -1, 1,  1]
w3    = [ -1,  1,  -1, 1,   1,  -1, -1,  1]
w4    = [ -1,  1, -1, -1,   1,  -1,  1,  1]




w_l1 = [w1,w2,w3,w4]


w5    = [ -1, -1, -1, 1]
# w6    = [ -1,  1,  1, 1]


def init(circuit, input, work):    
    for idx in range(len(input)):
        # if input[idx]<0.5:
        #     circuit.x(work[idx])
        y_v = input[idx].item()
        if y_v > 0:
            alpha = np.arccos(y_v)
        elif y_v < 0:
            alpha = np.pi - np.arccos(-y_v)
        else:
            alpha = np.pi/2
        circuit.ry(alpha,work[idx])
    circuit.barrier()
    

maxIndex = len(input)
        
def reset_qbits(q_set):
    for q in q_set:
        circuit.reset(q)
    circuit.barrier()

def SLP_2_encoding(circuit, q_in, q_en, input=[]):
    if len(input) != 0:
        init(circuit, input, q_in)
    encoder2_1(circuit, q_in, q_en)

def SLP_2_Uw(circuit, q_en, w, aux=[]):
    beg = len(circuit.data)
    generateU(circuit, w, q_en, aux)
    end = len(circuit.data)
    return beg, end

def SLP_4_encoding(circuit, q_in, q_en, input=[]):
    if len(input)!=0:        
        init(circuit,input,q_in)
        
    encoder_q_set = []
    for q in q_in:
        encoder_q_set.append(q)
    for q in q_en:
        encoder_q_set.append(q)
    encoder4_2(circuit,encoder_q_set,aux)

def SLP_4_Uw(circuit, q_en, q_out, w):
    
    beg = len(circuit.data)
    generateU(circuit, w,q_en,aux)
    end = len(circuit.data)
    
    circuit.barrier()
    for qbit in q_en[0:2]:
        circuit.h(qbit)
        circuit.x(qbit)
   
    circuit.ccx(q_en[0], q_en[1], q_out)
    circuit.barrier()
  
    return beg,end
def SLP_8_encoding(circuit, q_in, q_en, input=[]):
    if len(input)!=0:
        init(circuit, input, q_in)

    encoder_q_set = []
    for q in q_in:
        encoder_q_set.append(q)
    for q in q_en:
        encoder_q_set.append(q)
    encoder8_3(circuit,encoder_q_set,aux)
    
def SLP_8_Uw(circuit, q_en, q_out, w):    
    beg = len(circuit.data)
    generateU(circuit, w,q_en,aux)
    end = len(circuit.data)
    
    circuit.barrier()
    for qbit in q_en[0:3]:
        circuit.h(qbit)
        circuit.x(qbit)
   
    cccx(circuit, q_en[0], q_en[1], q_en[2], q_out, aux[0])
    circuit.barrier()
  
    return beg,end


def reverse_part_circuit(circuit,beg,end):
    
    for inst, qargs, cargs in reversed(circuit.data[beg:end]):
        circuit.data.append((inst.inverse(),qargs,cargs))
    



q_io = qk.QuantumRegister(8,"io")
q_en = qk.QuantumRegister(3,"encoded")
q_out = qk.QuantumRegister(4,"output")
c = qk.ClassicalRegister(1,"reg")
aux = qk.QuantumRegister(2,"aux")

circuit = qk.QuantumCircuit(q_io, q_en, q_out, aux, c)

for idx in range(len(w_l1)):
    SLP_8_encoding(circuit, q_io, q_en, input)
    reset_qbits(q_io)
    SLP_8_Uw(circuit, q_en, q_out[idx], w_l1[idx])
    circuit.barrier()
    reset_qbits(q_en)



SLP_4_encoding(circuit,q_out,q_en)
SLP_4_Uw(circuit,q_en,q_io[0],w5)
# for idx in range(len(w_l1)):
#     beg,end = SLP_8_Uw(circuit, q_en, q_io[idx], w_l1[idx])
#     reverse_part_circuit(circuit,beg,end)
#     circuit.barrier()


# print(circuit)

# 
# q_anc = q_io[0:4]
# encoder_q_set = []
# for q in q_anc:
#     encoder_q_set.append(q)
# encoder_q_set.append(q_en[0])
# encoder_q_set.append(q_en[1])
# encoder_ret_q = encoder4_2(circuit,encoder_q_set,aux)  
# generateU(circuit, w5,encoder_ret_q,aux)
# circuit.barrier()
# for qbit in encoder_ret_q:
#     circuit.h(qbit)
#     circuit.x(qbit)
# circuit.ccx(encoder_ret_q[0], encoder_ret_q[1], q_io[4])
# circuit.barrier()

circuit.measure(q_io[0],c)

print(circuit)

In [ ]:
qc_shots = 100
num_c_reg = 4

print("="*50)
print("Start simulation:")
start = time.time()        
iters = 1
counts = simulate(circuit,qc_shots,iters,False)

end = time.time()
qc_time = end - start


print("From QC:",counts)
print("Simulation elasped time:",qc_time)
    
def analyze(counts):
    mycount = {}
    for i in range(num_c_reg):
        mycount[i] = 0
    for k,v in counts.items():
        bits = len(k) 
        for i in range(bits):            
            if k[bits-1-i] == "1":
                if i in mycount.keys():
                    mycount[i] += v
                else:
                    mycount[i] = v
    return mycount,bits

# for k,v in counts[0].items():
#     print(k,v)
(mycount,bits) = analyze(counts[0])

for b in range(bits):
    print (b,float(mycount[b])/qc_shots)
    

In [5]:
def do_slp_via_th(input_ori,w_ori):
    p = input_ori
    d = 4*p*(1-p)
    e = (2*p-1)
    # e_sq = torch.tensor(1)
    w = torch.tensor(w_ori)
    
    sum_of_sq = (d+e.pow(2)).sum()
    sum_of_cross = 0
    for i in range(len(input_ori)):
        for j in range(i+1,len(input_ori)):
            sum_of_cross += w[i]*w[j]*e[i]*e[j]
    
    return (sum_of_sq+2*sum_of_cross)/(len(input_ori)**2) 

t_input = torch.tensor(input_ori)
t_w1 = torch.tensor(w1)
t_w2 = torch.tensor(w2)
t_w3 = torch.tensor(w3)
t_w4 = torch.tensor(w4)
t_w5 = torch.tensor(w5)
t_w_l1 = [t_w1,t_w2,t_w3,t_w4]
t_out_l1 = torch.tensor(np.ndarray([4]),dtype=torch.float32)
for idx in range(len(t_w_l1)):
    t_w = t_w_l1[idx]
    t_out_l1[idx] = do_slp_via_th(t_input,t_w)

t_out_l2 = do_slp_via_th(t_out_l1,t_w5)
print(t_out_l2)




NameError: name 'w1' is not defined

In [ ]:


class BinarizeF(torch.autograd.Function):
    @staticmethod
    def forward(cxt, input):
        output = input.new(input.size())
        output[input >= 0] = 1
        output[input < 0] = -1    
        return output
    @staticmethod
    def backward(cxt, grad_output):
        grad_input = grad_output.clone()
        return grad_input

# aliases
binarize = BinarizeF.apply



print(input)
t_input = input
t_w1 = torch.tensor(w1)
t_w2 = torch.tensor(w2)
t_w3 = torch.tensor(w3)
t_w4 = torch.tensor(w4)
t_w5 = torch.tensor(w5)
t_w6 = torch.tensor(w6)
t_w7 = torch.tensor(w7)
t_w8 = torch.tensor(w8)
t_w9 = torch.tensor(w9)


t_w_l1 = [t_w1,t_w2,t_w3,t_w4,t_w5,t_w6,t_w7,t_w8]
t_out_l1 = torch.tensor(np.ndarray([8]),dtype=torch.float32)
for idx in range(len(t_w_l1)):
    t_w = t_w_l1[idx]
    t_out_l1[idx] = (((t_input*t_w).sum()/float(len(t_input))).pow(2))
# print(((t_input*t_w3).sum()/len(t_input)).pow(2))
print(t_out_l1)
t_out_l1 = t_out_l1*2-1
print(t_out_l1)
print(t_w9)
print(((t_out_l1*t_w9).sum().float()/8).pow(2))


In [35]:

def do_slp_via_th(input_ori,w_ori):
    p = input_ori
    d = 4*p*(1-p)
    e = (2*p-1)
    # e_sq = torch.tensor(1)
    w = torch.tensor(w_ori)
    
    sum_of_sq = (d+e.pow(2)).sum()
    sum_of_cross = 0
    for i in range(len(input_ori)):
        for j in range(i+1,len(input_ori)):
            sum_of_cross += w[i]*w[j]*e[i]*e[j]
    
    return (sum_of_sq+2*sum_of_cross)/(len(input_ori)**2) 


# input_ori = [0.916,0.27,0.557,0.483]
input_ori = [0.8,0.6,0.1,0.3]
# input_ori = [0,1,0,0,1,0,0,1]
input = torch.tensor(input_ori)*2-1
w5    = [ 1, 1, 1, 1]

t_out_l2 = do_slp_via_th(torch.tensor(input_ori),torch.tensor(w5,dtype=torch.float))
print(t_out_l2)

tensor(0.1850)


/home/weiwen/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [28]:

q_io = qk.QuantumRegister(4,"io")
q_en = qk.QuantumRegister(2,"encoded")
q_out = qk.QuantumRegister(1,"output")
c = qk.ClassicalRegister(1,"reg")
aux = qk.QuantumRegister(2,"aux")

circuit = qk.QuantumCircuit(q_io, q_en, q_out, aux, c)

SLP_4_encoding(circuit,q_io,q_en,input)
reset_qbits(q_io)
SLP_4_Uw(circuit,q_en,q_out,w5)


circuit.measure(q_out,c)

qc_shots = 10000
num_c_reg = 4

print("="*50)
print("Start simulation:")
start = time.time()        
iters = 1
counts = simulate(circuit,qc_shots,iters,True)

end = time.time()
qc_time = end - start


print("From QC:",counts)
print("Simulation elasped time:",qc_time)
    


Start simulation:
              ┌────────────┐ ░                                                »
     io_0: |0>┤ Ry(0.9273) ├─░───■────■───────■───────────────────────────────»
              ├────────────┤ ░ ┌─┴─┐  │       │                               »
     io_1: |0>┤ Ry(1.3694) ├─░─┤ X ├──┼───■───┼────■───────■──────────────────»
              ├────────────┤ ░ └───┘┌─┴─┐ │   │    │       │                  »
     io_2: |0>┤ Ry(2.4981) ├─░──────┤ X ├─┼───┼────┼───────┼───■───■───────■──»
              ├────────────┤ ░      └───┘ │ ┌─┴─┐  │       │   │   │       │  »
     io_3: |0>┤ Ry(1.9823) ├─░────────────┼─┤ X ├──┼───────┼───┼───┼───────┼──»
              └────────────┘ ░ ┌───┐      │ └───┘  │       │   │   │       │  »
encoded_0: |0>───────────────░─┤ H ├──────┼────────┼───■───┼───■───■───────■──»
                             ░ ├───┤      │        │   │   │       │       │  »
encoded_1: |0>───────────────░─┤ H ├──────■────────■───┼───■───────┼───■───┼──»
                      

In [ ]:



input_ori = [0.9,0.1]
input = torch.tensor(input_ori)*2-1
w5    = [ 1, 0]

q_io = qk.QuantumRegister(2,"io")
q_en = qk.QuantumRegister(1,"encoded")
q_out = qk.QuantumRegister(1,"output")
c = qk.ClassicalRegister(1,"reg")


circuit = qk.QuantumCircuit(q_io, q_en, q_out, c)

SLP_2_encoding(circuit,q_io,q_en,input)
reset_qbits(q_io)
SLP_2_Uw(circuit,q_en,q_out,w5)


circuit.measure(q_out,c)

qc_shots = 10000
num_c_reg = 4

print("="*50)
print("Start simulation:")
start = time.time()        
iters = 1
counts = simulate(circuit,qc_shots,iters,True)

end = time.time()
qc_time = end - start


print("From QC:",counts)
print("Simulation elasped time:",qc_time)
    


In [ ]:

t_input = torch.tensor(input)
t_w = torch.tensor(w5)
print(t_input)
print(t_w)
print(((t_input*t_w).sum().float()/float(len(t_input))).pow(2))


In [ ]:
def print_c_qsm(c):
    print(c.qasm()[36:len(c.qasm())])

# q = [q_s[0][11],q_s[1][11],q_s[2][11],q_s[3][11]]
c = qk.ClassicalRegister(1,"reg")
aux = qk.QuantumRegister(2,"aux")
circuit = qk.QuantumCircuit(aux, c)
circuit.add_register(q_s[1])

circuit.measure(q_s[1][11],c[0])
print("="*50)
print("Start simulation")
start = time.time()        
iters = 1
counts = simulate(circuit,10000,iters,False)

end = time.time()
qc_time = end - start


print("From QC:",counts)
print("Simulation elasped time:",qc_time)
    

In [ ]:
q_in = qk.QuantumRegister(8,"input")
q_en = qk.QuantumRegister(3,"encoded")
q_anc = qk.QuantumRegister(4,"out1")
c = qk.ClassicalRegister(1,"reg")
aux = qk.QuantumRegister(2,"aux")

circuit = qk.QuantumCircuit(q_in, q_en, q_anc, aux, c)

idx = 0

generateU(circuit, w_l1[idx],q_en,aux)
circuit.barrier()
# 
# def SLP_8(circuit, q_in, q_en, q_out, input, w):
#     init(circuit, input, q_in)
# 
#     encoder_q_set = []
#     for q in q_in:
#         encoder_q_set.append(q)
#     for q in q_en:
#         encoder_q_set.append(q)
#     encoder_ret_q = encoder8_3(circuit,encoder_q_set,aux)    
#     generateU(circuit, w,encoder_ret_q,aux)
#     circuit.barrier()
#     for qbit in encoder_ret_q:
#         circuit.h(qbit)
#         circuit.x(qbit)
#         
#     cccx(circuit, encoder_ret_q[0], encoder_ret_q[1], encoder_ret_q[2], q_out, aux[0])
#     circuit.barrier()
# 
# SLP_8(circuit, q_in, q_en, q_anc[idx], input,w_l1[idx])
# reset_qbits(q_in)
# reset_qbits(q_en)
# print(circuit)
print(len(circuit.data))
circuit.data.extend(reversed(circuit.data[0:8]))
# print(circuit)

for gate in circuit.data:
    print(gate)
    print(gate[0].inverse())

# c2 = qk.QuantumCircuit.data(circuit.to_instruction(),q_in, q_en, q_anc, aux, c)
# print(c2)


In [ ]:


input_ori = [1,1,1,1,1,1,0,0]
# input_ori = [0,1,0,0,1,0,0,1]
input = torch.tensor(input_ori)*2-1


w1    = [ 1,  1, 1, 1,   1,  1, -1,  -1]
w2    = [ 1,  1,  1,  1,   1,  -1, -1,  -1]
w3    = [ 1,  1,  1, 1,   -1,  -1, -1,  -1]
w4    = [ 1,  1, 1, 1,   1,  -1,  1,  -1]
w_l1 = [w1,w2,w3,w4]
q_io = qk.QuantumRegister(8,"io")
q_en = qk.QuantumRegister(3,"encoded")

c = qk.ClassicalRegister(3,"reg")
aux = qk.QuantumRegister(2,"aux")

circuit = qk.QuantumCircuit(q_io, q_en, aux, c)

SLP_8_encoding(circuit, q_io, q_en, input)
reset_qbits(q_io)


idx = 0
beg,end = SLP_8_Uw(circuit, q_en, q_io[0:4], w_l1[idx])
# circuit.cx(q_io[idx],q_io[idx+1])
# reverse_part_circuit(circuit,beg,end)
circuit.barrier()

# for idx in range(len(w_l1)):
#     beg,end = SLP_8_Uw(circuit, q_en, q_io[idx], w_l1[idx])
#     reverse_part_circuit(circuit,beg,end)
#     circuit.barrier()
# reset_qbits(q_en)


for q in q_en:
    circuit.h(q)    
circuit.measure(q_en,c)

# circuit.measure(q_io[4],c[0])

print(circuit)


qc_shots = 1000
num_c_reg = 4

print("="*50)
print("Start simulation:")
start = time.time()        
iters = 1
counts = simulate(circuit,qc_shots,iters,False)

end = time.time()
qc_time = end - start


print("From QC:",counts)
print("Simulation elasped time:",qc_time)
    
def analyze(counts):
    mycount = {}
    for i in range(num_c_reg):
        mycount[i] = 0
    for k,v in counts.items():
        bits = len(k) 
        for i in range(bits):            
            if k[bits-1-i] == "1":
                if i in mycount.keys():
                    mycount[i] += v
                else:
                    mycount[i] = v
    return mycount,bits

# for k,v in counts[0].items():
#     print(k,v)
(mycount,bits) = analyze(counts[0])

for b in range(bits):
    print (b,float(mycount[b])/qc_shots)
    
    

In [ ]:
print(input)
t_input = input
t_w1 = torch.tensor(w1)
t_w2 = torch.tensor(w2)
t_w3 = torch.tensor(w3)
t_w4 = torch.tensor(w4)


t_w_l1 = [t_w1,t_w2,t_w3,t_w4]
t_out_l1 = torch.tensor(np.ndarray([4]),dtype=torch.float32)
for idx in range(len(t_w_l1)):
    t_w = t_w_l1[idx]
    
    t_out_l1[idx] = (((t_input*t_w).sum().float()/float(len(t_input))).pow(2))
# print(((t_input*t_w3).sum()/len(t_input)).pow(2))
print(t_out_l1)
# t_out_l1 = t_out_l1*2-1
# print(t_out_l1)